# Taylor series expansion of the kinetic energy operator for a triatomic molecule

In [25]:
import itertools

import jax
import jax.numpy as jnp
import numpy as np
from scipy.special import factorial

from vibrojet.keo import Gmat, batch_Gmat, com, eckart
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

In this example, we compute the Taylor series expansion coefficients for the kinetic-energy $G$-matrix of a triatomic molecule in terms of valence coordinates.

To achieve this, we first define a function that transforms an array of valence coordinates into an array of Cartesian coordinates of atoms. Molecular frame constraints can be conveniently applied using function decorators. For instance, the `com` decorator shifts the coordinates to the center of mass, while the `eckart` decorator rotates them into the Eckart frame.

The $G$-matrix can be computed for a single set of coordinate values using the `Gmat` function, or for a batch of valence coordinates using `batch_Gmat`.
The `Gmat` function outputs an array of shape (`ncoo`+3+3, `ncoo`+3+3), representing the elements of the $G$-matrix. The first `ncoo` elements correspond to vibrational (valence) coordinates, followed by three rotational coordinates and three translational coordinates.
The output of `batch_Gmat` extends this by introducing a leading dimension corresponding to the number of points in the batch, effectively computing the $G$-matrix for multiple coordinate sets simultaneously.

To compute derivatives or Taylor series expansion coefficients, we use the `deriv_list` function. This function requires three inputs: (1) the function to be differentiated (e.g., `Gmat`), (2) the reference valence coordinate values, and (3) a list of multi-indices specifying the integer exponents for each coordinate in the Taylor series expansion.

In [ ]:
# Masses of O, H, H atoms
masses = np.array([15.9994, 1.00782505, 1.00782505])

# Equilibrium values of valence coordinates
r1, r2, alpha = 0.958, 0.958, 1.824
x0 = jnp.array([r1, r2, alpha], dtype=jnp.float64)


# Valence-to-Cartesian coordinate transformation
#   input: array of three valence coordinates
#   output: array of shape (number of atoms, 3) containing Cartesian coordinates of atoms
# `com` shifts coordinates to the centre of mass
# `eckart` rotates coordinates to the Eckart frame
@eckart(x0, masses)
@com(masses)
def valence_to_cartesian(internal_coords):
    r1, r2, a = internal_coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.sin(a / 2), 0.0, r1 * jnp.cos(a / 2)],
            [-r2 * jnp.sin(a / 2), 0.0, r2 * jnp.cos(a / 2)],
        ]
    )


# Generate list of multi-indices specifying the integer exponents for each
# coordinate in the Taylor series expansion
max_order = 6  # max total expansion order
deriv_ind = [
    elem
    for elem in itertools.product(*[range(0, max_order + 1) for _ in range(len(x0))])
    if sum(elem) <= max_order
]

# Function for computing kinetic G-matrix for given masses of atoms
# and internal coordinates
func = lambda x: Gmat(x, masses, valence_to_cartesian)

# Compute Taylor series expansion coefficients
Gmat_coefs = deriv_list(func, deriv_ind, x0, if_taylor=True)

print(Gmat_coefs.shape)  # (number of Taylor terms, ncoo+3+3, ncoo+3+3)

Optionally, compare the results with more computationally intensive calculations by nesting multiple jacfwd calls for a selected element of $G$-matrix.

In [ ]:
from vibrojet._keo import Gmat as _Gmat

# Select element of G-matrix 
icoo = 2
jcoo = 2
func2 = lambda x: _Gmat(x, masses, valence_to_cartesian)[icoo, jcoo]


def jacfwd(x0, ind):
    f = func2
    for _ in range(sum(ind)):
        f = jax.jacfwd(f)
    i = sum([(i,) * o for i, o in enumerate(ind)], start=tuple())
    return f(x0)[i]


Gmat_coefs_jacfwd = np.array(
    [jacfwd(x0, ind) / np.prod(factorial(ind)) for ind in deriv_ind]
)


print("max difference:", np.max(np.abs(Gmat_coefs[:, icoo, jcoo] - Gmat_coefs_jacfwd)))
for i, ind in enumerate(deriv_ind):
    g1 = Gmat_coefs[i, icoo, jcoo]
    g2 = Gmat_coefs_jacfwd[i]
    print(ind, "%18.12f" % g1, "%18.12f" % g2, "%18.12f" % (g1 - g2))

max difference: 1.1537970578956447e-10
(0, 0, 0)    78.644737481988    78.644737481988     0.000000000000
(0, 0, 1)     4.445787034604     4.445787034604    -0.000000000000
(0, 0, 2)    -0.575189875515    -0.575189875516     0.000000000000
(0, 0, 3)    -0.740964505767    -0.740964505767     0.000000000000
(0, 0, 4)     0.047932489626     0.047932489626    -0.000000000000
(0, 0, 5)     0.037048225288     0.037048225288    -0.000000000000
(0, 0, 6)    -0.001597749654    -0.001597749654     0.000000000000
(0, 1, 0)   -82.092627851762   -82.092627851762     0.000000000000
(0, 1, 1)    -4.640696278292    -4.640696278292    -0.000000000000
(0, 1, 2)     0.600406968179     0.600406968179    -0.000000000000
(0, 1, 3)     0.773449379716     0.773449379715     0.000000000000
(0, 1, 4)    -0.050033914015    -0.050033914015     0.000000000000
(0, 1, 5)    -0.038672468984    -0.038672468986     0.000000000001
(0, 2, 0)   127.910787901318   127.910787901319    -0.000000000001
(0, 2, 1)     4.8441506

Example of computing $G$-matrix on a batch of coordinate values.

In [ ]:
# Generate grid of coordinates
r1_arr = np.linspace(r1 - 0.5, r1 + 1, 100)
r2_arr = np.linspace(r2 - 0.5, r2 + 1, 100)
alpha_arr = np.linspace(alpha - 40 * np.pi / 180, alpha + 40 * np.pi / 180, 100)
xa, xb, xc = np.meshgrid(r1_arr, r2_arr, alpha_arr, indexing="ij")
x = np.column_stack([xa.ravel(), xb.ravel(), xc.ravel()])

Gmat_vals = batch_Gmat(x, masses, valence_to_cartesian)

print(Gmat_vals.shape)  # (number of points, ncoo+3+3, ncoo+3+3)

(1000000, 9, 9)


Evaluate $G$-matrix from Taylor series

In [22]:
@jax.jit
def func_taylor(x, x0, ind, c):
    """Evaluates Taylor series: sum_i c[i] * (x - x0)**ind[i]"""
    dx = x - x0
    return jnp.sum(
        jnp.prod(dx[:, None, :] ** jnp.asarray(ind)[None, :, :], axis=-1) * c, axis=-1
    )

g_t = func_taylor(x, x0, deriv_ind, Gmat_coefs[:,2,2])

In [23]:
np.max(np.abs(Gmat_vals[:,2,2]-g_t))

np.float64(400.3255771598764)

In [24]:
np.sqrt(np.mean(np.square(Gmat_vals[:,2,2]-g_t)))

np.float64(67.41711952990254)